In [8]:
import cbpro
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yaml

yaml_filename = 'jay_cbpro_config.yaml'
my_yaml_file = open(f"../../{yaml_filename}")

yaml_dict = yaml.load(my_yaml_file, Loader=yaml.FullLoader)

locals().update(yaml_dict["cbpro"])

auth_client = cbpro.AuthenticatedClient(API_KEY, API_SECRET, API_PASS)
pub_client = cbpro.PublicClient()

In [55]:
"""

BTC, ETH, ADA, ZEC

"""
acc_info = {'BTC':None,
            'ETH':None,
            'ADA':None,
            'ZEC':None,
            'USDC':None}

for coin in acc_info.keys():
    temp = {'id':None, 'amount':None}
    for acc in auth_client.get_accounts():
        if acc['currency'] == coin:
            temp['id'] = acc['id']
            temp['amount'] = auth_client.get_account(acc['id'])['available']
            acc_info[coin] = temp

acc_info

{'BTC': {'id': '02552c36-438f-43e0-a2a1-d8dd0458693f', 'amount': '0.0003373'},
 'ETH': {'id': '02bda985-c6af-4d39-a44e-d13aac0a3526', 'amount': '0'},
 'ADA': {'id': '92772f4d-b63a-4ece-b376-4d456dc68072', 'amount': '11.42'},
 'ZEC': {'id': '7ae0c40f-7689-46ca-9c95-c907cde567f4', 'amount': '0'},
 'USDC': {'id': '99f1bd9f-6801-4b56-bfaf-32b0c89b88e4',
  'amount': '49.98788098910904'}}

In [56]:
"""

We are trading in an hour cadence

"""

price_info = {'BTC':None,
              'ETH':None,
              'ADA':None,
              'ZEC':None}

for coin in price_info.keys():
    temp = pd.DataFrame(pub_client.get_product_historic_rates(f'{coin}-USDC', granularity=3600),
                                    columns=['time','low','high','open','close','volume']).sort_values('time')
    temp['time'] = pd.to_datetime(temp['time'],unit='s')
    temp.set_index('time', inplace=True)
    price_info[coin] = temp

price_info['BTC'].tail()

,low,high,open,close,volume
time,,,,,
2021-09-10 21:00:00,44137.07,45268.87,45049.56,44578.77,59.168709
2021-09-10 22:00:00,44335.12,44873.52,44589.27,44444.45,68.803984
2021-09-10 23:00:00,44250.01,44881.73,44463.95,44846.53,62.706932
2021-09-11 00:00:00,44741.67,45252.44,44846.08,45208.13,43.650868
2021-09-11 01:00:00,45173.86,45484.70,45222.17,45441.66,20.227037


In [57]:
# helper functions for technical indicators
def _get_rsi(price, window=14):
    delta = price['close'].diff()
    up_days = delta.copy()
    up_days[delta<=0]=0.0
    down_days = abs(delta.copy())
    down_days[delta>0]=0.0
    RS_up = up_days.rolling(window).mean()
    RS_down = down_days.rolling(window).mean()
    rsi= 100-100/(1+RS_up/RS_down)
    return rsi


def _get_macd(price, window_1=12, window_2=26, os_window=9):
    ewm_12 = price['close'].ewm(span=window_1, adjust=False).mean()
    ewm_26 = price['close'].ewm(span=window_2, adjust=False).mean()
    macd = ewm_12 - ewm_26
    macd_ema = macd.ewm(span=os_window, adjust=False).mean()
    os = (macd - macd_ema)
    return macd, macd_ema, os

# dummy get_signal function
def get_signal(time, ind):
    
    # get indicators
    rsi, macd, macd_ema, os, vol = ind
    
    # get index
    time_60min = time
    time_60min_1 = time_60min - timedelta(minutes=60)
    time_60min = time_60min.strftime('%Y-%m-%d %H:00:00')
    time_60min_1 = time_60min_1.strftime('%Y-%m-%d %H:00:00')
    
    # dummy strategy
    if rsi.loc[time_60min] < 40 and (os.loc[time_60min_1] < 0 and os.loc[time_60min] > 0):
        return 1.0
    elif rsi.loc[time_60min] > 70 and (os.loc[time_60min_1] > 0 and os.loc[time_60min] < 0):
        return -1.0
    else:
        return 0

In [58]:
signal_info = {'BTC':None,
              'ETH':None,
              'ADA':None,
              'ZEC':None}

for coin in signal_info.keys():
    # get indicators
    price = price_info[coin]
    rsi = _get_rsi(price)
    macd, macd_ema, os = _get_macd(price)
    vol = price['volume']
    ind = rsi, macd, macd_ema, os, vol

    # get signal
    time = price_info[coin].index[-1]
    print('===========================')
    print(coin)
    print(time)
    print(rsi[-1])
    print(os[-1])
    signal = get_signal(time, ind)
    
    signal_info[coin] = signal

signal_info

BTC
2021-09-11 01:00:00
38.935941286504814
-21.90187246605592
ETH
2021-09-11 01:00:00
39.19500851690937
-2.663743878060714
ADA
2021-09-11 01:00:00
44.16135881104037
0.0036998390221217176
ZEC
2021-09-11 01:00:00
35.87819947043246
0.03693452759975768


{'BTC': 0, 'ETH': 0, 'ADA': 0, 'ZEC': 1.0}

In [59]:
sl_pct = 0.05
coins_unholding = len(acc_info.keys())

for coin in acc_info.keys():
    if float(acc_info[coin]['amount']) > 0 and coin != 'USDC':
        coins_unholding -= 1
        
print(f'Coins unholding: {coins_unholding}')

for coin in signal_info.keys():
    
    usdc_amount = round(float(acc_info['USDC']['amount'])/coins_unholding, 0)
    coin_amount = float(acc_info[coin]['amount'])
    signal = signal_info[coin]
    
    print('=============')
    print(f'Coin: {coin}')
    print(f'USDC: {usdc_amount}')
    
    # buy
    if signal > 0 and usdc_amount > 0.:
        print('buy')
        size = str(round(usdc_amount*signal,2))
        auth_client.place_market_order(product_id=f'{coin}-USDC',
                                       side='buy',
                                       funds=size)
        # stop order
        recent_price = next(pub_client.get_product_trades(product_id=f'{coin}-USDC'))['price']
        stoploss_price_trigger = str(round(float(recent_price)*(1-sl_pct),2))
        stoploss_price = str(round(float(recent_price)*(1-sl_pct-0.01),2))
        auth_client.place_order(product_id=f'{coin}-USDC', 
                                order_type='limit',
                                side='sell',
                                stop='loss',
                                stop_price=stoploss_price_trigger,
                                price=stoploss_price,
                                size=size)
        print(f'stoploss price: {stoploss_price}')

    # sell
    elif signal < 0 and coin_amount > 0.:
        print('sell')
        size = str(round(coin_amount*signal,2))
        auth_client.place_market_order(product_id=f'{coin}-USDC',
                                       side='sell',
                                       funds=size)
        # cancell stoploss order
        auth_client.cancel_all(product_id=f'{coin}-USDC')
    else:
        print('hold')
        pass

Coins unholding: 3
Coin: BTC
USDC: 17.0
hold
Coin: ETH
USDC: 17.0
hold
Coin: ADA
USDC: 17.0
hold
Coin: ZEC
USDC: 17.0
buy
stoploss price: 122.67


In [16]:
n = 3550
converters = 241

p = converters/n
q = 1-p

mu = p
se = np.sqrt(p*q/n)

confinf = 1.96*se

print(f'confidence interval: [{round((p-confinf)*100,2), round((p+confinf)*100,2)}]')

confidence interval: [(5.96, 7.62)]


In [29]:
from scipy.stats import chi2_contingency

temp = [['control', 3374, 197],
        ['Variant A', 3369, 227],
        ['Variant B', 3411, 265],
        ['Variant C', 3335, 207],
        ['Variant D', 3440, 241]]
temp = pd.DataFrame(np.array(temp))
temp.columns = ['group','total','converted']
temp.set_index('group', inplace=True)
temp['total'] = temp['total'].astype(int)
temp['converted'] = temp['converted'].astype(int)
temp

,total,converted
group,,
control,3374,197
Variant A,3369,227
Variant B,3411,265
Variant C,3335,207
Variant D,3440,241


In [30]:
chi2, p, dof, expected = chi2_contingency(temp.values)
print(p)

0.0328811464745038


In [51]:
from itertools import combinations

# gathering all combinations for post-hoc chi2
all_combinations = list(combinations(temp.index, 2))
p_vals = {}

for comb in all_combinations:
    # subset df into a dataframe containing only the pair "comb"
    df = temp[(temp.index == comb[0]) | (temp.index == comb[1])]
    # running chi2 test
    chi2, p, dof, ex = chi2_contingency(df, correction=False)
    p_vals[f'{comb[0]} & {comb[1]}'] = p
    
p_vals = pd.DataFrame.from_dict(p_vals, orient='index')
p_vals.columns = ['p_val']
p_vals

,p_val
control & Variant A,0.153293
control & Variant B,0.003198
control & Variant C,0.550732
control & Variant D,0.065528
Variant A & Variant B,0.128104
Variant A & Variant C,0.407660
Variant A & Variant D,0.683407
Variant B & Variant C,0.019043
Variant B & Variant D,0.262087
Variant C & Variant D,0.215617


In [52]:
from statsmodels.sandbox.stats.multicomp import multipletests

reject_list, corrected_p_vals = multipletests(p_vals.p_val, method='fdr_bh')[:2]

In [53]:
p_vals['corrected_p_val'] = corrected_p_vals
p_vals['reject'] = reject_list
p_vals

,p_val,corrected_p_val,reject
control & Variant A,0.153293,0.306587,False
control & Variant B,0.003198,0.031980,True
control & Variant C,0.550732,0.611925,False
control & Variant D,0.065528,0.218426,False
Variant A & Variant B,0.128104,0.306587,False
Variant A & Variant C,0.407660,0.509575,False
Variant A & Variant D,0.683407,0.683407,False
Variant B & Variant C,0.019043,0.095215,False
Variant B & Variant D,0.262087,0.374410,False
Variant C & Variant D,0.215617,0.359362,False
